In [ ]:
#A dataset is imported from hugging face hub
#it consists of multiple domain. Hence, we use the get_dataset_config_names() to find the subsets available
from datasets import get_dataset_config_names
xtreme_subset = get_dataset_config_names("xtreme")
print(f"xtreme has {len(xtreme_subset)} configurations")

In [ ]:
panx_dataset = [s for s in xtreme_subset if s.startswith("PAN")]
panx_dataset[:3]

In [ ]:
from datasets import load_dataset
load_dataset("xtreme",name="PAN-X.de")

In [ ]:
from collections import defaultdict
from datasets import DatasetDict
langs = ["de","fr","it","en"]
fracs = [0.629,0.229,0.084,0.059]
#return a datasetdict if a key doesn't exist
panx_ch = defaultdict(DatasetDict)

In [ ]:
#Pretokenization
for lang,frac in zip(langs,fracs):
    #load the monolingual corpus
    ds = load_dataset("xtreme",name=f"PAN-X.{lang}")
    #shuffe and downsample each split according to how they are spoken
    for split in ds:
        panx_ch[lang][split] = (
            ds[split].shuffle(seed=0).select(range(int(frac*ds[split].num_rows)))
        )

In [ ]:
import pandas as pd
pd.DataFrame({lang:[panx_ch[lang]["train"].num_rows]for lang in langs},index=["Number of training samples"])


In [ ]:
#Tokenization
element = panx_ch["de"]["train"][0]
for key,value in element.items():
    print(f"{key}:{value}")

In [ ]:
for key,value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

In [ ]:
#the NER tags
tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

In [ ]:
#create a int to str to create a new column for ner 
def create_tag_names(batch):
    return{"ner_tags_str":[tags.int2str(idx)for idx in batch["ner_tags"]]}

In [ ]:
panx_de = panx_ch["de"].map(create_tag_names)

In [ ]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"],de_example["ner_tags_str"]],['Tokens','Tags'])

In [ ]:
#count the frequency of each entity across a split
from collections import Counter
split2freqs = defaultdict(Counter)
for split,dataset in panx_de.items(): #check out the entire entity
    for row in dataset["ner_tags_str"]: #find the row of the dataset
        for tag in row: #then the tag
            if tag.startswith("B"): #then value of the key
                tag_type = tag.split("-")[-1]
                split2freqs[split][tag_type]+=1
pd.DataFrame.from_dict(split2freqs,orient="index")

In [ ]:
#tokenizer model
from transformers import AutoTokenizer
bert_model = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)


In [ ]:
text = "it is hell in here"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_token = xlmr_tokenizer(text).tokens()

In [ ]:
#post processing
bert_tokens

In [ ]:
xlmr_token

In [ ]:
#sentencePiece Tokenizer
sent_xlmr_token = "".join(xlmr_token).replace(u"\u2581"," ")

In [ ]:
sent_xlmr_token

In [ ]:
#custom model classification
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

In [ ]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig
    def __init__(self,config):
        super().__init__(config)
        self.num_labels = config.num_labels
        # Load model body
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        # Set up token classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # Load and initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, 
                labels=None, **kwargs):
        # Use model body to get encoder representations
        outputs = self.roberta(input_ids, attention_mask=attention_mask,
                               token_type_ids=token_type_ids, **kwargs)
        # Apply classifier to encoder representation
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        # Calculate losses
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        # Return model output object
        return TokenClassifierOutput(loss=loss, logits=logits, 
                                     hidden_states=outputs.hidden_states, 
                                     attentions=outputs.attentions)

In [ ]:
#provide additional meta data that can be used to label the NER
index2tag = {idx:tag for idx,tag in enumerate(tags.names)}
tag2index = {tag: idx for idx,tag in enumerate(tags.names)}

In [ ]:
#the model and certain constraints are downloaded
from transformers import AutoConfig
xlmr_config = AutoConfig.from_pretrained(xlmr_model_name,num_labels=tags.num_classes,id2label=index2tag,label2id=tag2index)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available () else "cpu")
xlmr_model = (XLMRobertaForTokenClassification.from_pretrained(xlmr_model_name,config=xlmr_config).to(device))

In [ ]:
input_ids = xlmr_tokenizer.encode(text,return_tensors="pt")
pd.DataFrame([xlmr_token,input_ids[0].numpy()],index=["Tokens","Input IDs"])

In [ ]:
outputs = xlmr_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs,dim=-1)
print(f"Number of tokens in sequence:{len(xlmr_token)}")
print(f"Shape of outputs: {outputs.shape}")

In [ ]:
def tag_text(text, tags, model, tokenizer):
 # Get tokens with special characters
 tokens = tokenizer(text).tokens()
 # Encode the sequence into IDs
 input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)
 # Get predictions as distribution over 7 possible classes
 outputs = model(input_ids)[0]
 # Take argmax to get most likely class per token
 predictions = torch.argmax(outputs, dim=2)
 # Convert to DataFrame
 preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
 return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

In [ ]:
#Collect words and tags as lists
words,labels = de_example["tokens"],de_example["ner_tags"]

In [ ]:
Tokenized_input = xlmr_tokenizer(de_example["tokens"],is_split_into_words=True)
tokens = xlmr_tokenizer.convert_ids_to_tokens(Tokenized_input["input_ids"])
pd.DataFrame([tokens],index=["Tokens"])

In [ ]:
words_id = Tokenized_input.word_ids()
pd.DataFrame([tokens,words_id],index=["Tokens","Words IDs"])

In [ ]:
#identify special tokens as [-100]
previous_word_idx = None
label_ids = []
for word_idx in words_id:
    if word_idx is None  or word_idx == previous_word_idx:
        label_ids.append(-100)
    elif word_idx != previous_word_idx:
        label_ids.append(labels[word_idx])
    previous_word_idx = word_idx

labels = [index2tag[l] if l != -100 else "IGN" for l in labels]
index = ["Token","Word IDs","Label IDs","Labels"]
pd.DataFrame([tokens,words_id,label_ids,labels],index=index)

In [ ]:
#Now we do same for a whole dataset
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(examples["tokens"],truncation=True,is_split_into_words=True)
    labels = []
    for idx,label    in enumerate(examples["ner_tags"]):
        words_id = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in words_id:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
#Iterate all the encoded inputs
def encode_panx_dataset(corpus):
    return corpus.map(tokenize_and_align_labels,batched=True,remove_columns=['langs','ner_tags','tokens'])

In [ ]:
panx_de_encoded = encode_panx_dataset(panx_ch["de"])

In [ ]:
from seqeval.metrics import classification_report
y_true = [["O", "O", "O", "B-MISC", "I-MISC", "I-MISC", "O"],["B-PER", "I-PER", "O"]]
y_pred = [["O", "O", "B-MISC", "I-MISC", "I-MISC", "I-MISC", "O"],["B-PER", "I-PER", "O"]]
print(classification_report(y_true,y_pred))


In [ ]:
#To convert the list to a seqeval list
import numpy as np
def align_predictions(predictions,label_ids):
    preds = np.argmax(predictions,axis=2)
    batch_size,seq_len = preds.shape
    labels_list,preds_list = [],[]
    for batch_idx in range(batch_size):
        example_labels,example_preds = [],[]
        #hide the special token during training
        for seq_idx in range(seq_len):
            if label_ids[batch_idx,seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])
        labels_list.append(example_labels)
        preds_list.append(example_preds)
    return preds_list,labels_list

In [ ]:
#fine tuning the XLM-Roberta transformer 
from transformers import TrainingArguments
num_epochs = 3
batch_size = 30
logging_steps = len(panx_de_encoded["train"]) // batch_size
model_name = f"{xlmr_model_name}-finetuned-panx-de"
training_args = TrainingArguments(
 output_dir=model_name, log_level="error", num_train_epochs=num_epochs,
 per_device_train_batch_size=batch_size,
 per_device_eval_batch_size=batch_size, evaluation_strategy="epoch",
 save_steps=1e6, weight_decay=0.01, disable_tqdm=False,
 logging_steps=logging_steps, push_to_hub=True)


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from seqeval.metrics import f1_score
def compute_metrics(eval_pred):
 y_pred, y_true = align_predictions(eval_pred.predictions,
 eval_pred.label_ids)
 return {"f1": f1_score(y_true, y_pred)}

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [ ]:
def model_init():
 return (XLMRobertaForTokenClassification
 .from_pretrained(xlmr_model_name, config=xlmr_config)
 .to(device))


In [ ]:
#pass all the information in an encoded dataset
from transformers import Trainer
trainer = Trainer(model_init=model_init, args=training_args,
 data_collator=data_collator, compute_metrics=compute_metrics,
 train_dataset=panx_de_encoded["train"],
 eval_dataset=panx_de_encoded["validation"],
 tokenizer=xlmr_tokenizer)

In [ ]:
#run a loop 
trainer.train() 


In [ ]:
trainer.push_to_hub(commit_message="training successful")

In [ ]:
text_de = "Jeff Dean ist ein Informatiker bei Google in Kalifornien"
tag_text(text_de,tags,trainer.model,xlmr_tokenizer)

In [ ]:
from torch.nn.functional import cross_entropy
def forward_pass_with_label(batch):
    # Convert dict of lists to list of dicts suitable for data collator
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    # Pad inputs and labels and put all tensors on device
    batch = data_collator(features)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    with torch.no_grad():
        # Pass data through model
        output = trainer.model(input_ids, attention_mask)
        # logit.size: [batch_size, sequence_length, classes]
        # Predict class with largest logit value on classes axis
        predicted_label = torch.argmax(output.logits, axis=-1).cpu().numpy()
        # Calculate loss per token after flattening batch dimension with view
    loss = cross_entropy(output.logits.view(-1, 7),
    labels.view(-1), reduction="none")
    # Unflatten batch dimension and convert to numpy array
    loss = loss.view(len(input_ids), -1).cpu().numpy()
    return {"loss":loss, "predicted_label": predicted_label}

In [ ]:
#use the function in a full dataset
valid_set = panx_de_encoded["validation"]
valid_set = valid_set.map(forward_pass_with_label, batched=True, batch_size=32)
df = valid_set.to_pandas()

In [ ]:
index2tag[-100] = "IGN"
df["input_tokens"] = df["input_ids"].apply(lambda x: xlmr_tokenizer.convert_ids_to_tokens(x))
df["predicted_label"] = df["predicted_label"].apply(lambda x: [index2tag[i] for i in x])
df["labels"] = df["labels"].apply(lambda x: [index2tag[i] for i in x])
df['loss'] = df.apply(lambda x: x['loss'][:len(x["input_ids"])],axis=1)
df['predicted_label'] = df.apply(lambda x: x['predicted_label'][:len(x['input_ids'])],axis=1)

df.head(4)

In [ ]:
#unpack the tokens of a single line into a lsit
df_tokens = df.apply(pd.Series.explode)
df_tokens = df_tokens.query("labels != 'IGN'")
df_tokens['loss'] = df_tokens['loss'].astype(float).round(2)
df_tokens.head(11)

In [ ]:
(
    df_tokens.groupby("input_tokens")[["loss"]]
    .agg(["count","mean","sum"])
    .droplevel(level=0,axis=1)#get rid of multi columns
    .sort_values(by="sum",ascending=False)
    .reset_index()
    .round(2)
    .head(10)
    .T
)

In [ ]:
(
    df_tokens.groupby("labels")[["loss"]]
    .agg(["count","mean","sum"])
    .droplevel(level=0,axis=1)#get rid of multi columns
    .sort_values(by="mean",ascending=False)
    .reset_index()
    .round(2)
    .T
)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
def plot_confusion_matrix(y_preds,y_true,labels):
    cm = confusion_matrix(y_true,y_preds,normalize="true")
    fig, ax = plt.subplots(figsize=(6,6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=labels)
    disp.plot(cmap="Blues",values_format=".2f",ax=ax,colorbar=False)
    plt.title("Normalised confusion matrix")
    plt.show
plot_confusion_matrix(df_tokens["labels"],df_tokens["predicted_label"],tags.names)

In [ ]:
def get_samples(df):
    for _,row in df.iterrow():
        labels,preds,tokens,losses = [],[],[],[]
        for i in enumerate(row["attention_mask"]):
            if i not in {0,len(row["attention_mask"])}:
                labels.append(row["labels"][i])
                preds.append(row["predicted_label"][i])
                tokens.append(row["input_tokens"][i])
                losses.append(f"{row['loss'][i]:.2f}")
        df_tmp = pd.DataFrame({"tokens":tokens,"labels":labels,"preds":preds,"losses":losses}).T
        yield df_tmp
df["total_loss"] = df["loss"].apply(sum)
df_tmp = df.sort_values(by="total_loss",ascending=False).head(3)
for sample in get_samples(df_tmp):
    display(sample)

In [ ]:
#error analysis across multiple languages
def get_f1_score(trainer,dataset):
    return trainer.predict(dataset).metrics["test_f1"]

In [ ]:
f1_scores = defaultdict(dict)
f1_scores["de"]["de"] = get_f1_score(trainer,panx_de_encoded["test"])
print(f"F1_score of [de] model on [de] dataset:{f1_scores['de']['de']:.3f}")

In [ ]:
text_fr = "Jeff Dean est informaticien chez Google en Californie"
tag_text(text_fr,tags,trainer.model,xlmr_tokenizer)

In [ ]:
#translating the entire deutsche dataset to french
def evaluate_lang_performance(lang,trainer):
    panx_ds = encode_panx_dataset(panx_ch["lang"])
    return get_f1_score(trainer,panx_ds["test"])

In [ ]:
f1_scores["de"]["fr"] = evaluate_lang_performance("fr",trainer)
print(f"F1_score for [de] model on [fr] is {f1_scores['de']['fr']:.3f}")

In [ ]:
f1_scores["de"]["it"] = evaluate_lang_performance("it", trainer)
print(f"F1-score of [de] model on [it] dataset: {f1_scores['de']['it']:.3f}")

In [ ]:
f1_scores["de"]["en"] = evaluate_lang_performance("en", trainer)
print(f"F1-score of [de] model on [en] dataset: {f1_scores['de']['en']:.3f}")

In [ ]:
#reconstruct our model for zero shot transfer
def train_on_subset(dataset, num_samples):
    train_ds = dataset["train"].shuffle(seed=42).select(range(num_samples))
    valid_ds = dataset["validation"]
    test_ds = dataset["test"]
    training_args.logging_steps = len(train_ds) // batch_size
    trainer = Trainer(model_init=model_init, args=training_args,
    data_collator=data_collator, compute_metrics=compute_metrics,
    train_dataset=train_ds, eval_dataset=valid_ds, tokenizer=xlmr_tokenizer)
    trainer.train()
    if training_args.push_to_hub:
        trainer.push_to_hub(commit_message="Training completed!")
    f1_score = get_f1_score(trainer, test_ds)
    return pd.DataFrame.from_dict({"num_samples": [len(train_ds)], "f1_score": [f1_score]})

In [ ]:
panx_fr_encoded = encode_panx_dataset(panx_ch["fr"])

In [ ]:
#we test the performance on 250 corpus
training_args.push_to_hub = False
metrics_df = train_on_subset(panx_fr_encoded, 250)
metrics_df

In [ ]:
#lets do this on 500-4000 corpus
for num_samples in [500, 1000, 2000, 4000]:
    metrics_df = metrics_df.append(train_on_subset(panx_fr_encoded, num_samples), ignore_index=True)

In [ ]:
fig,ax = plt.subplots()
ax.axhline(f1_scores["de"]["fr"],ls="--",color="r")
metrics_df.set_index("num_samples").plot(ax=ax)
plt.legend(["Zero-shot from de", "Fine-tuned on fr"], loc="lower right")
plt.ylim((0, 1))
plt.xlabel("Number of Training Samples")
plt.ylabel("F1 Score")
plt.show()

In [ ]:
#fine tuning on multiple languages at once (multilingual)
from datasets import concatenate_datasets
def concatenate_splits(corpora):
    multicorpus = DatasetDict()
    for split in corpora[0].keys:
        multicorpus[split] = concatenate_datasets([corpus[split] for corpus in corpora]).shuffle(seed=42)
    return multicorpus

In [ ]:
panx_de_fr_encoded = concatenate_splits([panx_de_encoded, panx_fr_encoded])

In [ ]:
training_args.logging_steps = len(panx_de_fr_encoded["train"]) // batch_size
training_args.push_to_hub = True
training_args.output_dir = "xlm-roberta-base-finetuned-panx-de-fr"


In [ ]:
trainer = Trainer(model_init=model_init, args=training_args,data_collator=data_collator, compute_metrics=compute_metrics,tokenizer=xlmr_tokenizer, train_dataset=panx_de_fr_encoded["train"],eval_dataset=panx_de_fr_encoded["validation"])

In [ ]:
trainer.train()
trainer.push_to_hub(commit_message="Training completed!")

In [ ]:
for lang in langs:
    f1 = evaluate_lang_performance(lang, trainer)
    print(f"F1-score of [de-fr] model on [{lang}] dataset: {f1:.3f}")

In [ ]:
# fine-tune on the remaining languages with our train_on_subset() function
corpora = [panx_de_encoded]
# Exclude German from iteration
for lang in langs[1:]:
    training_args.output_dir = f"xlm-roberta-base-finetuned-panx-{lang}"
    # Fine-tune on monolingual corpus
    ds_encoded = encode_panx_dataset(panx_ch[lang])
    metrics = train_on_subset(ds_encoded, ds_encoded["train"].num_rows)
    # Collect F1-scores in common dict
    f1_scores[lang][lang] = metrics["f1_score"][0]
    #Add monolingual corpus to list of corpora to concatenate
    corpora.append(ds_encoded)

In [ ]:
corpora_encoded = concatenate_splits(corpora)


In [ ]:
training_args.logging_steps = len(corpora_encoded["train"]) // batch_size
training_args.output_dir = "xlm-roberta-base-finetuned-panx-all"

In [ ]:
trainer = Trainer(model_init=model_init, args=training_args,data_collator=data_collator, compute_metrics=compute_metrics,tokenizer=xlmr_tokenizer, train_dataset=corpora_encoded["train"],eval_dataset=corpora_encoded["validation"])

In [ ]:
trainer.train()
trainer.push_to_hub(commit_message="Training completed!")

In [ ]:
for idx, lang in enumerate(langs):
    f1_scores["all"][lang] = get_f1_score(trainer, corpora[idx]["test"])

scores_data = {"de": f1_scores["de"],"each": {lang: f1_scores[lang][lang] for lang in langs},"all": f1_scores["all"]}

f1_scores_df = pd.DataFrame(scores_data).T.round(4)
f1_scores_df.rename_axis(index="Fine-tune on", columns="Evaluated on",inplace=True)
f1_scores_df